In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/wikipedia-movie-plots/wiki_movie_plots_deduped.csv


## 1. Extract the corpus: Movie Plots

In [2]:
df=pd.read_csv("/kaggle/input/wikipedia-movie-plots/wiki_movie_plots_deduped.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [4]:
corpus=df["Plot"][:15000].to_list()
len(corpus)
corpus[3]

'Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading "His Photographer" and "His Press Agent" respectively, follow him into the shot; the photographer sets up his camera. "Teddy" aims his rifle upward at the tree and fells what appears to be a common house cat, which he then proceeds to stab. "Teddy" holds his prize aloft, and the press agent takes notes. The second shot is taken in a slightly different part of the wood, on a path. "Teddy" rides the path on his horse towards the camera and out to the left of the shot, followed closely by the press agent and photographer, still dutifully holding their signs.'

## 2. Next up is cleaning

### Import the tokenizer

In [5]:
from nltk.tokenize import word_tokenize

### Convert all to lowercase

In [6]:
import string
for i in range(len(corpus)):
    temp=corpus[i].lower()
    corpus[i]=''.join([char for char in temp if char not in string.punctuation])



In [7]:
print(corpus[100])

grace roberts played by lea leland marries rancher edward smith who is revealed to be a neglectful viceridden spouse they have a daughter vivian dr franklin leonid samoloff whisks grace away from this unhappy life and they move to new york under aliases pretending to be married since surely smith would not agree to a divorce grace and franklin have a son walter milton s gould vivian gets sick however and grace and franklin return to save her somehow this reunion as smith had assumed grace to be dead causes the death of franklin this plot device frees grace to return to her fathers farm with both children1


### Remove all punctuations